In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import datasets
import joblib

df = pd.read_csv("C:/Users/Usuario/Documents/tuia/aprendizaje automatico/Unidad 6_ MLOps/ejemplo-streamlit-iris/weatherAUS.csv", delimiter=",")

def FunctionTransformer(df):
  dff=df.copy()
  localidades = ["SydneyAirport", "Sydney", "Canberra", "Melbourne", "MelbourneAirport" ]

  dff= dff[dff['Location'].isin(localidades)]
  dff = dff.dropna(subset=['RainTomorrow', 'RainfallTomorrow'])
 
  dff = df.drop('Location', axis=1)
  def rellenar(t):
      for columna in t.columns:

          # Verifica si la columna tiene datos faltantes
          if t[columna].isnull().any():

              # Si es numérica y continua, rellena con el promedio
              if pd.api.types.is_numeric_dtype(t[columna]) and not pd.api.types.is_integer_dtype(t[columna]):
                  t[columna].fillna(t[columna].mean(), inplace=True)

              # Si es numérica y discreta, rellena con la moda
              elif pd.api.types.is_numeric_dtype(t[columna]):
                  t[columna].fillna(t[columna].mode()[0], inplace=True)

            # Si es categórica, rellena con la moda
              else:
                  t[columna].fillna(t[columna].mode()[0], inplace=True)

      return t
  dff = rellenar(dff)
  dff =  dff.reset_index(drop = True)
  columnas_categoricas = dff.select_dtypes(exclude='number').columns
  le = LabelEncoder()
  for categorica in columnas_categoricas:
    dff[categorica] = le.fit_transform(dff[categorica])
  dff = dff.replace([np.inf, -np.inf], 0)
  dff.drop('Unnamed: 0', axis=1, inplace=True)
  dff.drop('Date', axis=1, inplace=True)


  return dff
dff=FunctionTransformer(df)


X = dff.drop(['RainTomorrow', 'RainfallTomorrow'], axis=1)
y = dff['RainTomorrow']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # esto simplemente imputa por la media.
    ('scaler', StandardScaler()),  # se realiza una estandarización.
    ('model', LogisticRegression())  # se entrena un modelo de regresión lineal.
])

pipeline.fit(X_train, y_train)

joblib.dump(pipeline, 'clima_pipeline.joblib')

['clima_pipeline.joblib']